## MODEL TRAINING

In [1]:
import os

In [2]:
%pwd

'e:\\Project\\MLOPs\\Chicken_disease_classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\Project\\MLOPs\\Chicken_disease_classification'

### 1. Entity for Model Training

In [5]:
from dataclasses import dataclass
from pathlib import Path

#This will be the entity variables
@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    base_model_path: Path
    trained_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int

### 2. Creating configuration for Model Training

In [11]:
from chicken_disease import *
from chicken_disease.constants import *
from chicken_disease.utils import (read_yaml, 
                                  create_directories)

In [7]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        """
        Initializes a ConfigurationManager instance.

        Args:
            config_filepath (str, optional): The file path to the configuration file.
                Defaults to CONFIG_FILE_PATH.
            params_filepath (str, optional): The file path to the parameters file.
                Defaults to PARAMS_FILE_PATH.
        """

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        """
        Retrieves the model trainer configuration.

        Returns:
            ModelTrainerConfig: The configuration object for the model trainer.
        """
        
        config = self.config.model_trainer
        
        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(root_dir=Path(config.root_dir),
                                                  base_model_path=Path(config.base_model_path),
                                                  trained_model_path=Path(config.trained_model_path),
                                                  params_image_size=self.params.IMAGE_SIZE,
                                                  params_learning_rate=self.params.LEARNING_RATE,
                                                  params_include_top=self.params.INCLUDE_TOP,
                                                  params_weights=self.params.WEIGHTS,
                                                  params_classes=self.params.CLASSES)

        return model_trainer_config


### 3. Train on the base model(VGG16)

In [8]:
import tensorflow as tf

In [9]:
class ModelTrainer:
    def __init__(self, 
                 config: ModelTrainerConfig,):
        """
        Initializes a ModelTrainer object.

        Args:
            config (ModelTrainerConfig): The configuration object containing parameters for the model trainer.
        """

        self.config = config
   
    def fetch_base_model(self):
        """
        Fetches the base model for training.

        This method loads the base model, specified in the configuration, using the provided parameters.
        The model is then saved to the specified path in the configuration.
        """

        self.model = tf.keras.applications.vgg16.VGG16(input_shape=self.config.params_image_size,
                                                       weights=self.config.params_weights,
                                                       include_top=self.config.params_include_top,)

        self.save_model(path=self.config.base_model_path, 
                        model=self.model,)
    
    @staticmethod
    def train_base_model(model, classes, freeze_all, freeze_till, learning_rate,):
        """
        Trains the base model.

        This method takes a base model, specified by the 'model' parameter, and performs training on it.
        The training can include freezing specific layers based on the provided parameters.
        The resulting trained model is compiled and returned.

        Args:
            model (tf.keras.Model): The base model to train.
            classes (int): The number of classes for classification.
            freeze_all (bool): Whether to freeze all layers of the model.
            freeze_till (int): Number of layers to freeze starting from the end. If None, no layers are frozen.
            learning_rate (float): The learning rate for the optimizer.

        Returns:
            tf.keras.Model: The trained model.
        """
        if freeze_all:
            for layer in model.layers:
                model.trainable = False

        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False

        flatten_in = tf.keras.layers.Flatten()(model.output)

        prediction = tf.keras.layers.Dense(units=classes,
                                           activation="softmax")(flatten_in)

        trained_model = tf.keras.models.Model(inputs=model.input,
                                           outputs=prediction)

        trained_model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
                              loss=tf.keras.losses.CategoricalCrossentropy(),
                              metrics=["accuracy"])

        trained_model.summary()
        return trained_model    

    def base_model_trainer(self):
        """
        Trains the base model specified in the configuration.

        This method trains the base model by calling the 'train_base_model' method with the appropriate parameters.
        The resulting trained model is saved to the specified path in the configuration.
        """

        self.trained_model = self.train_base_model(model=self.model,
                                                   classes=self.config.params_classes,
                                                   freeze_all=True,
                                                   freeze_till=None,
                                                   learning_rate=self.config.params_learning_rate,)

        self.save_model(path=self.config.trained_model_path, 
                        model=self.trained_model,)
    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        """
        Saves a Keras model to a specified path.

        Args:
            path (Path): The path to save the model.
            model (tf.keras.Model): The Keras model to save.
        """
        
        model.save(path)

### 4. Model Trainer pipeline

In [14]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config = model_trainer_config)
    model_trainer.fetch_base_model()
    model_trainer.base_model_trainer()
except Exception as e:
        logger.exception(CDCException(error_message=e, error_detail=sys))
        CDCException(error_message=e, error_detail=sys)

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   